In [1]:
import numpy as np
import pandas as pd
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
import sklearn
import keras, math
from  sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.callbacks import Callback
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation
import pickle
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from queue import Queue
%matplotlib inline

Using TensorFlow backend.
/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data_healthy = pickle.load(open('watsoniotp.healthy.pickle', 'rb'))
data_broken = pickle.load(open('watsoniotp.broken.pickle', 'rb'))

UnicodeDecodeError: 'ascii' codec can't decode byte 0xa4 in position 0: ordinal not in range(128)

In [5]:
data_healthy = pd.read_pickle('watsoniotp.healthy.pickle')
data_broken =pd.read_pickle('watsoniotp.broken.pickle')

In [8]:
print (data_healthy.shape)
print (data_broken.shape)

(3000, 3)
(3000, 3)


In [9]:
data_healthy = data_healthy.reshape(3000,3)
data_broken = data_broken.reshape(3000,3)

In [10]:
def scaleData(data):
    # normalize features
    scaler = MinMaxScaler(feature_range=(0, 1))
    return scaler.fit_transform(data)

In [11]:
data_healthy_scaled = scaleData(data_healthy)
data_broken_scaled = scaleData(data_broken)

In [12]:
timesteps = 10
dim = 3
samples = 300
data_healthy_scaled_reshaped = data_healthy_scaled

In [13]:
# design network

model = Sequential()
model.add(LSTM(50,input_shape=(timesteps,dim),return_sequences=False))
model.add(Dense(timesteps*dim))
model.compile(loss='mse', optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.95, decay=5e-4, nesterov=True))

In [18]:
from systemml.mllearn import Keras2DML

TabError: inconsistent use of tabs and spaces in indentation (estimators.py, line 922)

## Now use Keras2DML importer

In [17]:

epochs = 50
batch_size = 50
max_iter = int(epochs*math.ceil(samples/batch_size))
model = Keras2DML(spark, model, input_shape=(timesteps, dim, 1), batch_size=batch_size, max_iter=max_iter, test_interval=0, display=10)
model.set(perform_one_hot_encoding=False)
model.set(debug=True)


TabError: inconsistent use of tabs and spaces in indentation (estimators.py, line 922)

In [ ]:
def train(data):
    data = data.reshape(samples, timesteps*dim)
    model.fit(data, data) # epochs=50, batch_size=72, validation_data=(data, data), verbose=0, shuffle=False,callbacks=[LossHistory()])

In [ ]:
train(data_healthy_scaled)